In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import numpy as np
import datetime as datetime

In [2]:
df=pd.read_csv('../Datos/Originales/log2.csv')

In [3]:
df.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [4]:
df.dtypes

Source Port              int64
Destination Port         int64
NAT Source Port          int64
NAT Destination Port     int64
Action                  object
Bytes                    int64
Bytes Sent               int64
Bytes Received           int64
Packets                  int64
Elapsed Time (sec)       int64
pkts_sent                int64
pkts_received            int64
dtype: object

In [5]:
df_pequeno= df.sample(n=5000, random_state=42)

In [6]:
ELASTIC_PASSWORD='fpLcBH8a3VVZBc0w7OnL'

client=Elasticsearch(
    'https://localhost:9200',
    ca_certs='/home/marta/elasticsearch-8.4.3/config/certs/http_ca.crt',
    basic_auth=('elastic', ELASTIC_PASSWORD)
)

client.info()

ObjectApiResponse({'name': 'marta-VirtualBox', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'W_W6tmGfTuauRs68H-lxMg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
for i in range(len(df_pequeno)):
    doc=dict()
    data=dict(df_pequeno.iloc[i])
    doc['Source_port']=int(data['Source Port'])
    doc["Destination_port"] = int(data['Destination Port'])
    doc['NAT_source_port']=int(data['NAT Source Port'])
    doc["NAT_destination_port"] = int(data['NAT Destination Port'])
    doc['Action']=data['Action']
    doc["Bytes"] = int(data['Bytes'])
    doc['Bytes_sent']=int(data['Bytes Sent'])
    doc["Bytes_received"] = int(data['Bytes Received'])
    doc['Packets']=int(data['Packets'])
    doc["Elapsed_Time_(sec)"] = int(data['Elapsed Time (sec)'])
    doc['pkts_sent']=int(data['pkts_sent'])
    doc["pkts_received"] = int(data['pkts_received'])
    client.index(index='log2', document=doc, id=i+1)